# piezo

This jupyter notebook walks you through the basic functionality of the Classes provided using the resistance catalogue `config/NEJM2018-RSU-catalogue-H37rV_v3.csv`. This is the catalogue published in the Supplement of this paper

The CRyPTIC Consortium, 100000 Genomes Project. Prediction of Susceptibility to First-Line Tuberculosis Drugs by DNA Sequencing. N Engl J Med 2018;379:1403–1415. [http://doi.org/10.1056/NEJMoa1800474](doi:10.1056/NEJMoa1800474)

It is important to note that this catalogue is relative to version 3 of the H37rV GenBank reference (the version published in the paper was with respect to version 2).

The differences are
* some default rows have been added which describe the underlying assumptions such as any synoymous mutation is susceptible (in other words, all the logic is now contained in the catalogue, not in the code)
* the mutations are formatted according to the General Ontology for Antimicrobial Resistance Catalogues (GOARC) which is described in the NOMENCLATURE.md file in the repo and also [here](http://fowlerlab.org/2018/11/25/goarc-a-general-ontology-for-antimicrobial-resistance-catalogues/)

Philip W Fowler

8 Mar 2019

In [1]:
# we will need pandas so we can have a look at the Resistance Catalogue which is stored as a CSV
import pandas
import piezo

In [5]:
nejm_catalogue=pandas.read_csv("config/NEJM2018-RSU-catalogue-H37rV_v3.csv")

In [6]:
nejm_catalogue[:5]

,DRUG,GENE,MUTATION,POSITION,GENE_TYPE,VARIANT_AFFECTS,VARIANT_TYPE,GENBANK_REFERENCE,NEJM2018_PREDICTION,NEJM2018_PHYLOSNP
0,EMB,embA,2723_indel,2723,GENE,CDS,INDEL,NC_000962.3,S,False
1,EMB,embA,A1015T,1015,GENE,CDS,SNP,NC_000962.3,S,False
2,EMB,embA,A1016S,1016,GENE,CDS,SNP,NC_000962.3,S,False
3,EMB,embA,A109T,109,GENE,CDS,SNP,NC_000962.3,S,False
4,EMB,embA,A201T,201,GENE,CDS,SNP,NC_000962.3,S,False


In [9]:
nejm_catalogue["GENE_MUTATION"]=nejm_catalogue["GENE"]+"_"+nejm_catalogue["MUTATION"]

Let's check a well-known mutation that confers resistance to rifampicin

In [10]:
nejm_catalogue.loc[nejm_catalogue.GENE_MUTATION=="rpoB_S450L"]

,DRUG,GENE,MUTATION,POSITION,GENE_TYPE,VARIANT_AFFECTS,VARIANT_TYPE,GENBANK_REFERENCE,NEJM2018_PREDICTION,NEJM2018_PHYLOSNP,GENE_MUTATION
1042,RIF,rpoB,S450L,450,GENE,CDS,SNP,NC_000962.3,R,False,rpoB_S450L


These are the default rules that have been added for this gene

In [11]:
nejm_catalogue.loc[(nejm_catalogue.GENE=="rpoB") & (nejm_catalogue.POSITION=="*")]

,DRUG,GENE,MUTATION,POSITION,GENE_TYPE,VARIANT_AFFECTS,VARIANT_TYPE,GENBANK_REFERENCE,NEJM2018_PREDICTION,NEJM2018_PHYLOSNP,GENE_MUTATION
1076,RIF,rpoB,*=,*,GENE,CDS,SNP,NC_000962.3,S,NaN,rpoB_*=
1077,RIF,rpoB,*?,*,GENE,CDS,SNP,NC_000962.3,U,NaN,rpoB_*?
1078,RIF,rpoB,-*?,*,GENE,PROM,SNP,NC_000962.3,U,NaN,rpoB_-*?
1079,RIF,rpoB,*_indel,*,GENE,CDS,INDEL,NC_000962.3,U,NaN,rpoB_*_indel
1080,RIF,rpoB,-*_indel,*,GENE,PROM,INDEL,NC_000962.3,U,NaN,rpoB_-*_indel


`*=` means any synonymous mutation in the protein coding sequence (CDS). Here `*` is reserved for 'any position' and `!` for the Stop codon. `*?` is any non-synoymous mutation in the CDS and `*_indel` is any insertion or deletion at any position in the CDS. Finally, `-*?` and `-*_indel` mean any nonsynoymous mutation or insertion/deletion in the promoter, respectively

## Instantiating a catalogue

You need to specify the catalogue CSV, the matching GenBank file and the name of the catalogue (since a single CSV can have multiple columns associated with a prediction: there MUST be a column called "LID2015B_PREDICTION" otherwise the code will complain).



In [39]:
cat=piezo.ResistanceCatalogue(input_file="config/NEJM2018-RSU-catalogue-H37rV_v3.csv",
                              genbank_file="config/H37rV_v3.gbk",
                              catalogue_name="NEJM2018")

## Predicting the effect of a mutation

Now it is a simple as using the `predict()` method. This method takes two arguments, (1) `gene_mutation` which requires mutations in the form `gene_mutation` e.g. `rpoB_S450L` or `rpoB_1300_ins` and (2) `verbose` which if set to `True` the code will print out to STDOUT the rules that have been met for that mutation and their associated priorties. The prediction with the highest priority is then chosen.

In [15]:
cat.predict(gene_mutation='rpoB_S450W')

{'RIF': 'R'}

A dictionary is returned since a single genetic variant can affect multiple drugs.

These are the rows in the catalogue that will be considered for a mutation at `rpoB_S450` 

In [17]:
cat.resistance_catalogue.loc[(cat.resistance_catalogue.GENE=='rpoB') & 
                             (cat.resistance_catalogue.POSITION.isin(['*','450'])) &
                             (cat.resistance_catalogue.VARIANT_TYPE=='SNP') & 
                             (cat.resistance_catalogue.VARIANT_AFFECTS=='CDS')]

,DRUG,GENE,MUTATION,POSITION,GENE_TYPE,VARIANT_AFFECTS,VARIANT_TYPE,GENBANK_REFERENCE,NEJM2018_PREDICTION,NEJM2018_PHYLOSNP
1040,RIF,rpoB,S450?,450,GENE,CDS,SNP,NC_000962.3,R,False
1041,RIF,rpoB,S450F,450,GENE,CDS,SNP,NC_000962.3,R,False
1042,RIF,rpoB,S450L,450,GENE,CDS,SNP,NC_000962.3,R,False
1043,RIF,rpoB,S450Q,450,GENE,CDS,SNP,NC_000962.3,R,False
1044,RIF,rpoB,S450W,450,GENE,CDS,SNP,NC_000962.3,R,False
1045,RIF,rpoB,S450Y,450,GENE,CDS,SNP,NC_000962.3,R,False
1076,RIF,rpoB,*=,*,GENE,CDS,SNP,NC_000962.3,S,NaN
1077,RIF,rpoB,*?,*,GENE,CDS,SNP,NC_000962.3,U,NaN


So, let's repeat the prediction and see which rules are hit

In [18]:
cat.predict(gene_mutation='rpoB_S450L',verbose=True)

RIF rpoB_S450L
2. U nonsyn SNP at any position in the CDS or PROM
3. R nonsyn SNP at specified position in the CDS
4. R exact SNP match
----------------------


{'RIF': 'R'}

As you can see the mutation first matches `rpoB_*?` (any non-synoymous mutation in the protein coding region of rpoB) and this has a priority of 2. It then matches an exact rule for `rpoB_S450L` which has a priority of 4. The predicted phenotype of the rule with the highest priority is returned, in this case `R` as expected.

This hierarchical approach will become increasingly important, for example, the `fabG1_L203L` synonymous mutation is (unusually) predicted to confer resistance to INH, however, simply adding a specific row to the catalogue will naturally ensure the specific rule overrides the default rule that any synoymous mutation is assumed to be susceptible.

## Validation

The code uses a separate repository called [gemucator](https://github.com/philipwfowler/gemucator). You instantiate a copy of the class with the correct (i.e. same) GenBank file.

This all happens "under the hood" in the piezo module, but it is useful to see the extent of validation that is performed on each mutation.

In [19]:
from gemucator import gemucator

In [20]:
tb=gemucator(genbank_file="config/H37rV_v3.gbk")

The two methods most useful for validation are `valid_gene` and `valid_mutation`. 

In [21]:
tb.valid_gene("rpoB")

True

Let's try a gene that doesn't exist

In [22]:
tb.valid_gene("rpoD")

False

In [23]:
tb.valid_mutation("rpoB_S450L")

True

and a mutation that doesn't (this checks the REF amino acid against the GenBank file which isn't often done but should be correct, as otherwise there may be an inconsistency between the catalogue and the GenBank File)

In [24]:
tb.valid_mutation("rpoB_T450L")

False

It also insists that all amino acids are UPPERCASE and are drawn from the correct 20 amino acid alphabet and nucleotides are lowercase `[a,c,t,g]`

In [25]:
tb.valid_mutation("rpoB_J450L")

AssertionError: J is not an amino acid!

In [26]:
tb.valid_mutation("rpoB_c-15t")

True

In [27]:
tb.valid_mutation("rpoB_d-15t")

AssertionError: d is not a nucleotide!

Now let's see it in action inside `piezo`

In [28]:
cat.predict(gene_mutation='rpoB_J450L')

AssertionError: J is not an amino acid!

In [29]:
cat.predict(gene_mutation='rpoB_D450L')

AssertionError: gene exists but rpoB_D450L is badly formed; check the reference amino acid or nucleotide!

In [30]:
cat.predict(gene_mutation='rpoD_S450L')

AssertionError: rpoD does not exist in the specified GENBANK file!

It should be able to parse all the styles of mutations that are defined by GOARC, e.g.

In [31]:
cat.predict(gene_mutation='rpoB_S450?')

{'RIF': 'R'}

In GOARC, INDELs are put into a hierachy, but it parses them all

In [32]:
cat.predict(gene_mutation='katG_100_indel')

{'INH': 'U'}

In [33]:
cat.predict(gene_mutation='katG_100_ins')

{'INH': 'U'}

Importantly for this catalogue, it understands frame shifts!

In [44]:
cat.predict(gene_mutation='pncA_2_ins_3',verbose=True)

PZA pncA_2_ins_3
1. U any insertion or deletion in the CDS or PROM
4. R any frameshifting insertion or deletion in the CDS
----------------------


{'PZA': 'R'}

In [45]:
cat.predict(gene_mutation='pncA_2_ins_4',verbose=True)

PZA pncA_2_ins_4
1. U any insertion or deletion in the CDS or PROM
----------------------


{'PZA': 'U'}

In [46]:
cat.predict(gene_mutation='pncA_2_ins_12',verbose=True)

PZA pncA_2_ins_12
1. U any insertion or deletion in the CDS or PROM
4. R any frameshifting insertion or deletion in the CDS
----------------------


{'PZA': 'R'}

In [34]:
cat.predict(gene_mutation='katG_100_del')

{'INH': 'U'}

In [35]:
cat.predict(gene_mutation='katG_100_ins_4')

{'INH': 'U'}

In [36]:
cat.predict(gene_mutation='katG_100_ins_actg')

{'INH': 'U'}